In [2]:
import pandas as pd

In [3]:
movies=pd.read_csv('IMDB movies.csv')

C:\Users\Harman\.conda\envs\PDSH\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Drop irrelevant columns

movies.drop(columns=['usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics','duration','country','language','date_published','title'],inplace=True)

movies.shape

In [5]:
movies.head(5)

,imdb_title_id,original_title,year,genre,director,writer,production_company,actors,description,avg_vote,votes,budget
0,tt0000009,Miss Jerry,1894,Romance,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250
2,tt0001892,Den sorte drøm,1911,Drama,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN
3,tt0002101,Cleopatra,1912,"Drama, History",Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy","Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN
5,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,"Biography, Drama",Sidney Olcott,Gene Gauntier,Kalem Company,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,484,NaN
6,tt0002423,Madame DuBarry,1919,"Biography, Drama, Romance",Ernst Lubitsch,"Norbert Falk, Hanns Kräly",Projektions-AG Union (PAGU),"Pola Negri, Emil Jannings, Harry Liedtke, Edua...","The story of Madame DuBarry, the mistress of L...",6.8,753,NaN
7,tt0002445,Quo Vadis?,1913,"Drama, History",Enrico Guazzoni,"Henryk Sienkiewicz, Enrico Guazzoni",Società Italiana Cines,"Amleto Novelli, Gustavo Serena, Carlo Cattaneo...","An epic Italian film ""Quo Vadis"" influenced ma...",6.2,273,ITL 45000
8,tt0002452,Independenta Romaniei,1912,"History, War","Aristide Demetriade, Grigore Brezeanu","Aristide Demetriade, Petre Liciu",Societatea Filmului de Arta Leon Popescu,"Aristide Demetriade, Constanta Demetriade, Con...",The movie depicts the Romanian War of Independ...,6.7,198,ROL 400000
9,tt0002461,Richard III,1912,Drama,"André Calmettes, James Keane","James Keane, William Shakespeare",Le Film d'Art,"Robert Gemp, Frederick Warde, Albert Gardner, ...",Richard of Gloucester uses manipulation and mu...,5.5,225,$ 30000


In [26]:
df=movies[movies.original_title == 'Inception']
df

In [1]:
import tmdbsimple as ts
ts.API_KEY='YOUR_API_KEY'

In [61]:
res=ts.Find(id='tt1375666').info(external_source='imdb_id')
res